# Sampler Primitive - Code Laboratory

**Section 5: Sampler** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `StatevectorSampler()` | `StatevectorSampler()` | Sampler instance | Local simulation |
| `SamplerV2()` | `SamplerV2(mode=backend)` | Sampler instance | Real hardware |
| `sampler.run()` | `run(pubs, shots=None)` | `PrimitiveJob` | Execute circuits |
| `job.result()` | `result()` | `PrimitiveResult` | Get results |
| `result[i].data.meas.get_counts()` | `get_counts()` | `dict[str, int]` | Aggregated counts |
| `result[i].data.meas.get_bitstrings()` | `get_bitstrings()` | `list[str]` | Individual shots |

---

## 📚 Topics in This Notebook

| Topic | Cell Range | Key Mnemonics |
|-------|------------|---------------|
| StatevectorSampler basics | 3-7 | "S needs M" |
| PUB Format | 8-11 | "TiL", "CPS" |
| Result Extraction | 12-15 | "DrMeGC", "0-D-M-G" |
| Twirling Options | 16-17 | "GOMO" |
| Dynamical Decoupling | 18-19 | "DD = Defend During Downtime" |
| Sampler vs Estimator | 20-21 | "Sampler Samples, Estimator Expects" |
| Challenges | 22-26 | All concepts |

---

In [ ]:
"""
Qiskit Code Laboratory - Sampler Primitive
==========================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.circuit import Parameter

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def verify_counts(counts, expected_states, tolerance=0.1):
    """Verify that counts match expected states within tolerance."""
    total = sum(counts.values())
    for state in expected_states:
        prob = counts.get(state, 0) / total
        if prob < tolerance:
            return False
    return True

def show_distribution(counts, title=""):
    """Display measurement distribution with histogram."""
    total = sum(counts.values())
    print(f"{title}")
    for state, count in sorted(counts.items()):
        prob = count / total
        bar = '█' * int(prob * 30)
        print(f"  |{state}⟩: {prob:.3f} {bar}")

def create_bell_circuit():
    """Create standard Bell state circuit with measurements."""
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    return qc

print("✅ Environment ready")
print("✅ Utility functions: verify_counts(), show_distribution(), create_bell_circuit()")

---

## `StatevectorSampler` / `sampler.run()`

### Signature
```python
StatevectorSampler()
sampler.run(pubs: list[tuple], shots: int = None) -> PrimitiveJob
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `pubs` | `list[tuple]` | Yes | - | List of PUBs: `[(circuit, params, shots)]` |
| `shots` | `int` | No | Implementation-dependent | Number of measurement shots |

### Returns
`PrimitiveJob` - Job object with `.result()` method

### PUB Format (Primitive Unified Bloc)
| Format | Example | Description |
|--------|---------|-------------|
| `[(qc,)]` | Basic single circuit | Note trailing comma! |
| `[(qc, params)]` | With parameter values | `params` is list of values |
| `[(qc, None, shots)]` | Custom shots | Per-circuit shots |
| `[(qc, params, shots)]` | Full format | Circuit, Params, Shots |

### See Also
- README Section: [Topic 2: PUB Format](./README.md#-topic-2-pub-format-primitive-unified-blocs)

In [ ]:
# ============================================================
# StatevectorSampler - BASIC USAGE
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

# Create circuit WITH measurements (required for Sampler!)
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print("Bell State Circuit:")
print(qc.draw())

# Create Sampler and run
sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=1024)  # Note: [(qc,)] with trailing comma!

# Get results
result = job.result()
counts = result[0].data.c.get_counts()

print(f"\nMeasurement counts: {counts}")
print(f"Total shots: {sum(counts.values())}")

# Verification
assert '00' in counts or '11' in counts, "Bell state should have 00 or 11"
assert sum(counts.values()) == 1024, "Should have 1024 total shots"
print("✅ Basic Sampler usage verified")

In [ ]:
# ============================================================
# StatevectorSampler - PARAMETER VARIATIONS
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

sampler = StatevectorSampler()

# Variation 1: Multiple circuits in single call
qc1 = QuantumCircuit(2, 2)
qc1.h(0)
qc1.cx(0, 1)
qc1.measure([0, 1], [0, 1])

qc2 = QuantumCircuit(1, 1)
qc2.x(0)
qc2.measure(0, 0)

print("Variation 1: Multiple circuits")
job = sampler.run([(qc1,), (qc2,)], shots=512)
result = job.result()
print(f"Circuit 1 (Bell): {result[0].data.c.get_counts()}")
print(f"Circuit 2 (X|0⟩): {result[1].data.c.get_counts()}")

# Variation 2: Custom shots per circuit
print("\nVariation 2: Different shots per circuit")
job = sampler.run([(qc1, None, 100), (qc2, None, 500)])
result = job.result()
counts1 = result[0].data.c.get_counts()
counts2 = result[1].data.c.get_counts()
print(f"Circuit 1 (100 shots): {sum(counts1.values())} total")
print(f"Circuit 2 (500 shots): {sum(counts2.values())} total")

# Variation 3: Parameterized circuits
from qiskit.circuit import Parameter
import numpy as np

theta = Parameter('θ')
qc_param = QuantumCircuit(1, 1)
qc_param.ry(theta, 0)
qc_param.measure(0, 0)

print("\nVariation 3: Parameterized circuit sweep")
angles = [0, np.pi/4, np.pi/2, np.pi]
pubs = [(qc_param, [angle]) for angle in angles]
job = sampler.run(pubs, shots=256)
result = job.result()

for i, angle in enumerate(angles):
    counts = result[i].data.c.get_counts()
    p1 = counts.get('1', 0) / 256
    print(f"θ={angle:.2f}: P(|1⟩)={p1:.2f}")

print("✅ Parameter variations verified")

In [ ]:
# ============================================================
# StatevectorSampler - EDGE CASES & ERROR HANDLING
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

sampler = StatevectorSampler()

# Edge Case 1: Single shot
print("Edge Case 1: Single shot")
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)
job = sampler.run([(qc,)], shots=1)
counts = job.result()[0].data.c.get_counts()
print(f"Single shot result: {counts}")
assert sum(counts.values()) == 1, "Should have exactly 1 shot"

# Edge Case 2: Large number of shots
print("\nEdge Case 2: Large shots (10000)")
job = sampler.run([(qc,)], shots=10000)
counts = job.result()[0].data.c.get_counts()
total = sum(counts.values())
print(f"Total shots: {total}")
p0 = counts.get('0', 0) / total
print(f"P(|0⟩) = {p0:.3f} (expected ~0.5)")

# Error Case 1: Circuit without measurements
print("\n⚠️ ERROR DEMONSTRATION: No measurements")
try:
    qc_no_meas = QuantumCircuit(1)
    qc_no_meas.h(0)
    # No measure() call!
    job = sampler.run([(qc_no_meas,)])
    result = job.result()
    print(f"Result (may be empty): {result}")
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")
    print("✅ Sampler requires measurements!")

# Error Case 2: Wrong PUB format
print("\n⚠️ ERROR DEMONSTRATION: Wrong PUB format")
try:
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    qc.measure(0, 0)
    job = sampler.run([(qc)])  # Missing comma!
except Exception as e:
    print(f"Error: {type(e).__name__}")
    print("✅ Must use [(qc,)] with trailing comma!")

In [ ]:
# ============================================================
# StatevectorSampler - VERIFICATION PATTERN
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

def test_sampler():
    """Test suite for StatevectorSampler."""
    sampler = StatevectorSampler()
    
    # Test 1: Basic execution
    qc = QuantumCircuit(1, 1)
    qc.x(0)
    qc.measure(0, 0)
    job = sampler.run([(qc,)], shots=100)
    counts = job.result()[0].data.c.get_counts()
    assert counts.get('1', 0) == 100, "X gate should give all |1⟩"
    print("✅ Test 1: Basic execution passed")
    
    # Test 2: Bell state distribution
    qc_bell = QuantumCircuit(2, 2)
    qc_bell.h(0)
    qc_bell.cx(0, 1)
    qc_bell.measure([0, 1], [0, 1])
    job = sampler.run([(qc_bell,)], shots=1000)
    counts = job.result()[0].data.c.get_counts()
    # Bell state should only have 00 and 11
    for state in counts.keys():
        assert state in ['00', '11'], f"Unexpected state {state}"
    print("✅ Test 2: Bell state distribution passed")
    
    # Test 3: Multiple circuit indexing
    qc1 = QuantumCircuit(1, 1); qc1.measure(0, 0)  # All 0
    qc2 = QuantumCircuit(1, 1); qc2.x(0); qc2.measure(0, 0)  # All 1
    job = sampler.run([(qc1,), (qc2,)], shots=50)
    result = job.result()
    c1 = result[0].data.c.get_counts()
    c2 = result[1].data.c.get_counts()
    assert c1.get('0', 0) == 50, "Circuit 1 should be all 0"
    assert c2.get('1', 0) == 50, "Circuit 2 should be all 1"
    print("✅ Test 3: Multiple circuit indexing passed")
    
    print("\n✅ All Sampler tests passed!")

test_sampler()

---

## `result[i].data.register.get_counts()` / `get_bitstrings()`

### Signature
```python
result[i].data.register_name.get_counts() -> dict[str, int]
result[i].data.register_name.get_bitstrings() -> list[str]
```

### Result Access Chain
| Component | Description |
|-----------|-------------|
| `result[i]` | i-th circuit's result (0-indexed) |
| `.data` | Data container with BitArrays |
| `.register_name` | Classical register name (default: "c" or "meas") |
| `.get_counts()` | Aggregated histogram |
| `.get_bitstrings()` | Individual shot outcomes |

### See Also
- README Section: [Topic 3: Result Extraction](./README.md#-topic-3-result-extraction)

In [ ]:
# ============================================================
# Result Extraction - BASIC USAGE
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

# Create and run Bell circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=100)
result = job.result()

# Access result for first circuit
pub_result = result[0]

# Method 1: get_counts() - histogram
counts = pub_result.data.c.get_counts()
print("get_counts() - Aggregated histogram:")
print(f"  Type: {type(counts)}")
print(f"  Value: {counts}")
print(f"  Unique states: {len(counts)}")

# Method 2: get_bitstrings() - individual shots
bitstrings = pub_result.data.c.get_bitstrings()
print("\nget_bitstrings() - Individual shots:")
print(f"  Type: {type(bitstrings)}")
print(f"  First 5: {bitstrings[:5]}")
print(f"  Total: {len(bitstrings)} (equals shots)")

# Verification
assert len(counts) <= len(bitstrings), "counts has unique states only"
assert len(bitstrings) == 100, "bitstrings has one per shot"
print("\n✅ Result extraction verified")

In [ ]:
# ============================================================
# Result Extraction - CUSTOM REGISTER NAMES
# ============================================================

from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.primitives import StatevectorSampler

sampler = StatevectorSampler()

# Default register name (using measure())
qc1 = QuantumCircuit(2, 2)
qc1.h(0)
qc1.cx(0, 1)
qc1.measure([0, 1], [0, 1])

job1 = sampler.run([(qc1,)], shots=100)
result1 = job1.result()[0]
# Default classical register is 'c'
counts1 = result1.data.c.get_counts()
print(f"Default register 'c': {counts1}")

# With measure_all() - register name is 'meas'
qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.cx(0, 1)
qc2.measure_all()

job2 = sampler.run([(qc2,)], shots=100)
result2 = job2.result()[0]
counts2 = result2.data.meas.get_counts()
print(f"measure_all() register 'meas': {counts2}")

# Custom register name
cr = ClassicalRegister(2, 'my_result')
qc3 = QuantumCircuit(2)
qc3.add_register(cr)
qc3.h(0)
qc3.cx(0, 1)
qc3.measure([0, 1], cr)

job3 = sampler.run([(qc3,)], shots=100)
result3 = job3.result()[0]
counts3 = result3.data.my_result.get_counts()
print(f"Custom register 'my_result': {counts3}")

print("\n✅ Register name must match what you used!")
print("   measure() default → 'c'")
print("   measure_all() → 'meas'")
print("   Custom → your name")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: PUB Format - Trailing Comma
# This is what the exam might test!
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

print("⚠️ TRAP: PUB Format - Trailing Comma")
print("=" * 50)

qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)
sampler = StatevectorSampler()

# Show Python tuple behavior
print("\n🔍 Python tuple behavior:")
print(f"type((qc))  = {type((qc))}")    # NOT a tuple!
print(f"type((qc,)) = {type((qc,))}")   # Tuple with trailing comma

# ❌ WRONG - Common mistakes
print("\n❌ WRONG approaches:")
print("sampler.run(qc)         # Not wrapped at all")
print("sampler.run([qc])       # List, not tuple")
print("sampler.run([(qc)])     # Parentheses, not tuple!")

# ✅ CORRECT
print("\n✅ CORRECT approach:")
job = sampler.run([(qc,)])  # Trailing comma makes it a tuple!
counts = job.result()[0].data.c.get_counts()
print(f"sampler.run([(qc,)]) → {counts}")

print("\n💡 Remember: TiL = Tuple in List")
print("   Outer: List [...] of PUBs")
print("   Inner: Tuple (...,) with trailing comma!")
print("   See README.md for mnemonic!")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Sampler REQUIRES Measurements
# This is what the exam might test!
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

print("⚠️ TRAP: Sampler REQUIRES Measurements")
print("=" * 50)

sampler = StatevectorSampler()

# ❌ WRONG - No measurements
print("\n❌ WRONG: Circuit without measurements")
qc_wrong = QuantumCircuit(2)
qc_wrong.h(0)
qc_wrong.cx(0, 1)
print(qc_wrong.draw())
print("↑ No measure() calls!")

try:
    job = sampler.run([(qc_wrong,)])
    result = job.result()
    # May succeed but give empty/wrong results
    print(f"Result may be empty or error: {result}")
except Exception as e:
    print(f"Error: {e}")

# ✅ CORRECT - With measurements
print("\n✅ CORRECT: Circuit WITH measurements")
qc_correct = QuantumCircuit(2, 2)
qc_correct.h(0)
qc_correct.cx(0, 1)
qc_correct.measure([0, 1], [0, 1])
print(qc_correct.draw())

job = sampler.run([(qc_correct,)], shots=100)
counts = job.result()[0].data.c.get_counts()
print(f"Result: {counts}")

print("\n🔍 THE DIFFERENCE:")
print("Sampler → needs measure() (samples measurement outcomes)")
print("Estimator → NO measure() (calculates expectation values)")
print("\n💡 Remember: S needs M = Sampler needs Measures")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Result Access Chain
# This is what the exam might test!
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

print("⚠️ TRAP: Result Access Chain")
print("=" * 50)

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=100)
result = job.result()

# ❌ WRONG patterns
print("\n❌ WRONG approaches:")
print("result.counts                    # No .counts attribute")
print("result.get_counts()              # Result is not a dict")
print("result[0].get_counts()           # Missing .data.register")
print("result.data.c.get_counts()       # Missing [0] index")

# ✅ CORRECT
print("\n✅ CORRECT approach:")
counts = result[0].data.c.get_counts()
print(f"result[0].data.c.get_counts() → {counts}")

# Show the chain
print("\n🔍 CHAIN BREAKDOWN:")
print(f"result              → {type(result)}")
print(f"result[0]           → {type(result[0])}")
print(f"result[0].data      → {type(result[0].data)}")
print(f"result[0].data.c    → BitArray")
print(f"result[0].data.c.get_counts() → {counts}")

print("\n💡 Remember: DrMeGC = Data.Register.Method.Get_Counts")
print("   Or: 0-D-M-G = [0].data.meas.get_counts()")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Bit Ordering (LSB)
# This is what the exam might test!
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

print("⚠️ TRAP: Bit Ordering (LSB = Least Significant Bit)")
print("=" * 50)

# Create circuit: X on qubit 0 only
qc = QuantumCircuit(3, 3)
qc.x(0)  # Only qubit 0 is |1⟩
qc.measure([0, 1, 2], [0, 1, 2])

sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=10)
bitstrings = job.result()[0].data.c.get_bitstrings()

print(f"\nCircuit: X on qubit 0 only")
print(f"Bitstrings: {bitstrings[:3]}...")

# Show bit ordering
print("\n🔍 Bit Ordering:")
bs = bitstrings[0]
print(f"Bitstring: '{bs}'")
print(f"           ↑↑↑")
print(f"           ││└─ qubit 0 (rightmost = LSB)")
print(f"           │└── qubit 1")
print(f"           └─── qubit 2 (leftmost = MSB)")

# Accessing individual qubits
print(f"\nAccessing qubit values from '{bs}':")
print(f"  qubit 0 = bs[-1] = '{bs[-1]}'  (rightmost)")
print(f"  qubit 1 = bs[-2] = '{bs[-2]}'")
print(f"  qubit 2 = bs[-3] = '{bs[-3]}'  (leftmost)")

# Common mistake
print("\n❌ WRONG: Thinking leftmost is qubit 0")
print("✅ CORRECT: Rightmost is qubit 0 (LSB ordering)")
print("\n💡 Remember: LSB = Last (rightmost) is Start Bit (qubit 0)")

In [ ]:
# ============================================================
# COMPARISON: get_counts() vs get_bitstrings()
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

print("=" * 60)
print("COMPARING: get_counts() vs get_bitstrings()")
print("=" * 60)

# Create Bell state
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=1000)
bit_array = job.result()[0].data.c

# Method 1: get_counts()
counts = bit_array.get_counts()
print(f"\nget_counts():")
print(f"  Returns: dict[str, int]")
print(f"  Value: {counts}")
print(f"  Length: {len(counts)} (unique outcomes only)")

# Method 2: get_bitstrings()  
bitstrings = bit_array.get_bitstrings()
print(f"\nget_bitstrings():")
print(f"  Returns: list[str]")
print(f"  First 5: {bitstrings[:5]}")
print(f"  Length: {len(bitstrings)} (one per shot)")

# Key difference
print(f"\n🔍 KEY DIFFERENCE:")
print(f"  get_counts()     → {len(counts)} items (memory efficient)")
print(f"  get_bitstrings() → {len(bitstrings)} items (full data)")

print("\n✅ Use get_counts() for statistics/histograms")
print("✅ Use get_bitstrings() for shot-by-shot analysis")

---

## Twirling Options (Error Mitigation)

### Key Concept
**Pauli twirling** converts coherent noise into stochastic noise by randomly applying Pauli gates around two-qubit gates.

### Options Reference
| Option | Default (Sampler) | Default (Estimator) | Description |
|--------|-------------------|---------------------|-------------|
| `enable_gates` | `False` | `False` | Twirl 2-qubit gates |
| `enable_measure` | `False` ⚠️ | `True` ⚠️ | Twirl measurements |
| `num_randomizations` | 32 | 32 | Number of random variants |
| `strategy` | `"active-accum"` | `"active-accum"` | Which qubits to twirl |

### Strategies
| Strategy | Description |
|----------|-------------|
| `"active"` | Only qubits with gates in current layer |
| `"active-accum"` | All qubits used up to current layer (default) |
| `"active-circuit"` | All qubits used anywhere in circuit |
| `"all"` | ALL qubits in circuit |

### Mnemonic: **"GOMO"** = **G**ates **O**ff, **M**easure **O**ff (Sampler defaults)

### See Also
- README Section: [Topic 4: Twirling Options](./README.md#-topic-4-twirling-options)

In [ ]:
# ============================================================
# TWIRLING OPTIONS - CONFIGURATION PATTERNS
# ============================================================
# NOTE: This demonstrates Runtime Sampler options
# For local StatevectorSampler, twirling is not applicable

print("=" * 60)
print("TWIRLING OPTIONS - Runtime Sampler Configuration")
print("=" * 60)

# Pattern showing how to configure twirling (for runtime)
# This is conceptual - requires qiskit_ibm_runtime

print("""
# ==================== RUNTIME CONFIGURATION ====================
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')
sampler = Sampler(mode=backend)

# Gate twirling (twirl 2-qubit gates)
sampler.options.twirling.enable_gates = True

# Measurement twirling
sampler.options.twirling.enable_measure = True  # False by default for Sampler!

# Number of random variants
sampler.options.twirling.num_randomizations = 32

# Shots per randomization (total = num_rand × shots_per)
sampler.options.twirling.shots_per_randomization = 100

# Strategy options: "active", "active-accum", "active-circuit", "all"
sampler.options.twirling.strategy = "active-accum"
""")

# ============================================================
# ⚠️ TRAP DEMONSTRATION: Sampler vs Estimator Defaults
# ============================================================

print("\n" + "=" * 60)
print("⚠️ EXAM TRAP: Sampler vs Estimator Twirling DEFAULTS")
print("=" * 60)

print("""
┌────────────────┬─────────────────┬──────────────────┐
│ Option         │ SAMPLER Default │ ESTIMATOR Default│
├────────────────┼─────────────────┼──────────────────┤
│ enable_gates   │     False       │      False       │
│ enable_measure │     False ⚠️    │      True ⚠️     │
│ strategy       │  "active-accum" │  "active-accum"  │
└────────────────┴─────────────────┴──────────────────┘

⚠️ KEY TRAP: enable_measure defaults are DIFFERENT!
   - Sampler:   enable_measure = False (off by default)
   - Estimator: enable_measure = True  (on by default)
""")

print("\n💡 Mnemonic: GOMO = Gates Off, Measure Off (Sampler defaults)")
print("   Estimator has Measure ON by default!")
print("\n✅ This is a common exam question - know the defaults!")

---

## Dynamical Decoupling (DD)

### Key Concept
**Dynamical Decoupling** inserts pulse sequences on idle qubits to suppress errors from environmental interactions.

### DD Sequences
| Sequence | Pattern | Description |
|----------|---------|-------------|
| `"XX"` | X - X | Simple, default |
| `"XpXm"` | X+ - X- | Robust to pulse errors |
| `"XY4"` | X - Y - X - Y | **Superior suppression** |

### Configuration
| Option | Values | Description |
|--------|--------|-------------|
| `enable` | `True/False` | Enable DD |
| `sequence_type` | `"XX"`, `"XpXm"`, `"XY4"` | Pulse sequence |
| `extra_slack_distribution` | `"middle"`, `"edges"` | Pulse placement |
| `scheduling_method` | `"alap"`, `"asap"` | When to insert |

### Mnemonic: **"DD = Defend During Downtime"**
- DD only helps when qubits are **idle**
- No idle time = no benefit from DD

### See Also
- README Section: [Topic 5: Dynamical Decoupling](./README.md#-topic-5-dynamical-decoupling)

In [ ]:
# ============================================================
# DYNAMICAL DECOUPLING - CONFIGURATION & DEMONSTRATION
# ============================================================

print("=" * 60)
print("DYNAMICAL DECOUPLING - Runtime Sampler Configuration")
print("=" * 60)

print("""
# ==================== RUNTIME CONFIGURATION ====================
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')
sampler = Sampler(mode=backend)

# Enable dynamical decoupling
sampler.options.dynamical_decoupling.enable = True

# Choose sequence type
sampler.options.dynamical_decoupling.sequence_type = "XY4"  # Best suppression
# Options: "XX" (default, simple), "XpXm" (robust), "XY4" (superior)

# How to distribute pulses in idle periods
sampler.options.dynamical_decoupling.extra_slack_distribution = "middle"
# Options: "middle", "edges"

# Scheduling method
sampler.options.dynamical_decoupling.scheduling_method = "alap"
# Options: "alap" (As Late As Possible), "asap" (As Soon As Possible)
""")

# ============================================================
# ⚠️ TRAP DEMONSTRATION: DD Only Helps with Idle Qubits
# ============================================================

from qiskit import QuantumCircuit

print("\n" + "=" * 60)
print("⚠️ EXAM TRAP: DD Only Helps with IDLE Qubits")
print("=" * 60)

# Circuit WITH idle periods - DD helps
print("\n✅ DD HELPS: Circuit with idle qubits")
qc_gaps = QuantumCircuit(3, 3)
qc_gaps.h(0)
qc_gaps.barrier()
qc_gaps.cx(1, 2)  # Qubit 0 is IDLE during this!
qc_gaps.barrier()
qc_gaps.cx(0, 1)
qc_gaps.measure_all()

print("Circuit with idle qubit (q0 during cx(1,2)):")
print(qc_gaps.draw())
print("↑ DD pulses would be inserted on q0 during the gap")

# Circuit WITHOUT idle periods - DD doesn't help much
print("\n⚠️ DD MINIMAL EFFECT: Circuit with no gaps")
qc_no_gaps = QuantumCircuit(2, 2)
qc_no_gaps.h([0, 1])
qc_no_gaps.cx(0, 1)
qc_no_gaps.measure_all()

print("Circuit with no idle periods:")
print(qc_no_gaps.draw())
print("↑ No gaps = DD has minimal effect")

print("\n💡 Mnemonic: DD = Defend During Downtime")
print("   No downtime (idle qubits) = No defense needed")
print("\n✅ Know when DD helps and when it doesn't for the exam!")

---

## Sampler vs Estimator Comparison

### Quick Reference
| Aspect | Sampler | Estimator |
|--------|---------|-----------|
| **Purpose** | Measurement outcomes | Expectation values |
| **Returns** | Counts, bitstrings | ⟨ψ\|O\|ψ⟩ values |
| **Requires** | Measurements in circuit | Observable (NO measurements!) |
| **Twirling Measure Default** | `False` | `True` |

### PUB Format Comparison
| Primitive | Format | Example |
|-----------|--------|---------|
| Sampler | `(circuit, params, shots)` | `[(qc_with_meas,)]` |
| Estimator | `(circuit, observable, params, precision)` | `[(qc_no_meas, SparsePauliOp('ZZ'))]` |

### When to Use Which
- **Sampler**: Counts, probabilities, Grover's, QML classification
- **Estimator**: VQE, QAOA, chemistry, observable measurements

### See Also
- README Section: [Topic 6: Sampler vs Estimator](./README.md#-topic-6-sampler-vs-estimator-comparison)

In [ ]:
# ============================================================
# SAMPLER vs ESTIMATOR - Side-by-Side Comparison
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

print("=" * 70)
print("SAMPLER vs ESTIMATOR - Side-by-Side Demonstration")
print("=" * 70)

# Same quantum state: Bell state
# But different circuits for each primitive!

# ============================================================
# SAMPLER: Requires measurements
# ============================================================
print("\n" + "─" * 35)
print("SAMPLER: Measurement Outcomes")
print("─" * 35)

qc_sampler = QuantumCircuit(2, 2)
qc_sampler.h(0)
qc_sampler.cx(0, 1)
qc_sampler.measure([0, 1], [0, 1])  # REQUIRED!

print("Circuit for Sampler (WITH measurements):")
print(qc_sampler.draw())

sampler = StatevectorSampler()
job_s = sampler.run([(qc_sampler,)], shots=1000)
counts = job_s.result()[0].data.c.get_counts()
print(f"Sampler Output: {counts}")
print("↳ Returns: dict of measurement counts")

# ============================================================
# ESTIMATOR: No measurements, uses observables
# ============================================================
print("\n" + "─" * 35)
print("ESTIMATOR: Expectation Values")
print("─" * 35)

qc_estimator = QuantumCircuit(2)
qc_estimator.h(0)
qc_estimator.cx(0, 1)
# NO measurements!

print("Circuit for Estimator (NO measurements):")
print(qc_estimator.draw())

observable = SparsePauliOp('ZZ')
print(f"Observable: {observable}")

estimator = StatevectorEstimator()
job_e = estimator.run([(qc_estimator, observable)])
evs = job_e.result()[0].data.evs
print(f"Estimator Output: ⟨ZZ⟩ = {evs}")
print("↳ Returns: expectation value (float)")

# ============================================================
# KEY DIFFERENCES SUMMARY
# ============================================================
print("\n" + "=" * 70)
print("🔑 KEY DIFFERENCES")
print("=" * 70)

print("""
┌──────────────────┬────────────────────────┬─────────────────────────┐
│     Aspect       │       SAMPLER          │       ESTIMATOR         │
├──────────────────┼────────────────────────┼─────────────────────────┤
│ Circuit requires │ measure() / measure_all│ NO measurements!        │
│ Additional input │ (none)                 │ Observable (SparsePauliOp)│
│ Returns          │ Counts dict            │ Expectation value float │
│ Result access    │ .data.meas.get_counts()│ .data.evs               │
│ Twirling measure │ Default: False         │ Default: True           │
│ Use case         │ Grover, QML, sampling  │ VQE, QAOA, chemistry    │
└──────────────────┴────────────────────────┴─────────────────────────┘

Mnemonic: "Sampler Samples (counts), Estimator Expects (⟨O⟩)"
""")

print("✅ Know which primitive to use for what task on the exam!")

In [ ]:
# ============================================================
# CHALLENGE 1: Verify GHZ State
# ============================================================
# Task: Create a 3-qubit GHZ state and verify with Sampler
# Expected: Only |000⟩ and |111⟩ outcomes (roughly 50% each)
# Hint: GHZ = H on q0, then CX chain
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

def challenge_ghz():
    """
    Create a 3-qubit GHZ state: (|000⟩ + |111⟩)/√2
    
    Returns:
        dict: Measurement counts
    """
    # YOUR CODE HERE
    qc = QuantumCircuit(3, 3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.measure([0, 1, 2], [0, 1, 2])
    
    sampler = StatevectorSampler()
    job = sampler.run([(qc,)], shots=1000)
    counts = job.result()[0].data.c.get_counts()
    return counts

# Test your solution
counts = challenge_ghz()
print("Your GHZ results:")
for state, count in sorted(counts.items()):
    print(f"  |{state}⟩: {count/1000:.1%}")

# Verification
valid_states = all(s in ['000', '111'] for s in counts.keys())
balanced = abs(counts.get('000', 0) - counts.get('111', 0)) < 200

if valid_states and balanced:
    print("✅ Challenge 1 PASSED! Valid GHZ state")
else:
    print("❌ Challenge 1 FAILED - check your circuit")

In [ ]:
# ============================================================
# CHALLENGE 2: Parameter Sweep - Ry Gate
# ============================================================
# Task: Sweep Ry(θ) from 0 to π and verify P(|1⟩) = sin²(θ/2)
# Expected: P(|1⟩) increases from 0 to 1 as θ goes from 0 to π
# Hint: Use parameterized circuit with multiple PUBs
# ============================================================

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorSampler
import numpy as np

def challenge_param_sweep():
    """
    Sweep Ry(θ)|0⟩ and return P(|1⟩) for each angle.
    
    Returns:
        list: [(angle, p1), ...] for angles 0, π/4, π/2, 3π/4, π
    """
    # YOUR CODE HERE
    theta = Parameter('θ')
    qc = QuantumCircuit(1, 1)
    qc.ry(theta, 0)
    qc.measure(0, 0)
    
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi]
    pubs = [(qc, [angle]) for angle in angles]
    
    sampler = StatevectorSampler()
    job = sampler.run(pubs, shots=1000)
    results = job.result()
    
    data = []
    for i, angle in enumerate(angles):
        counts = results[i].data.c.get_counts()
        p1 = counts.get('1', 0) / 1000
        data.append((angle, p1))
    return data

# Test your solution
results = challenge_param_sweep()
print("Parameter sweep results:")
print(f"{'θ':^10} | {'P(|1⟩)':^10} | {'Theory':^10} | {'Match':^6}")
print("-" * 45)

all_pass = True
for angle, p1 in results:
    theory = np.sin(angle/2)**2
    match = abs(p1 - theory) < 0.1
    all_pass = all_pass and match
    status = "✅" if match else "❌"
    print(f"{angle:^10.3f} | {p1:^10.3f} | {theory:^10.3f} | {status:^6}")

if all_pass:
    print("\n✅ Challenge 2 PASSED! P(|1⟩) = sin²(θ/2) verified")
else:
    print("\n❌ Challenge 2 FAILED - check your implementation")

In [ ]:
# ============================================================
# CHALLENGE 3: Multi-Circuit Batch Execution
# ============================================================
# Task: Create 3 circuits, run as batch, extract results by index
# Expected: Each circuit should give deterministic result
# Hint: Use [(qc1,), (qc2,), (qc3,)] format
# ============================================================

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

def challenge_batch():
    """
    Create 3 deterministic circuits and run as batch.
    
    Returns:
        list: [counts1, counts2, counts3]
    """
    # YOUR CODE HERE
    # Circuit 1: Should give all |0⟩
    qc1 = QuantumCircuit(1, 1)
    qc1.measure(0, 0)
    
    # Circuit 2: Should give all |1⟩
    qc2 = QuantumCircuit(1, 1)
    qc2.x(0)
    qc2.measure(0, 0)
    
    # Circuit 3: Should give all |11⟩
    qc3 = QuantumCircuit(2, 2)
    qc3.x([0, 1])
    qc3.measure([0, 1], [0, 1])
    
    # Run as batch
    sampler = StatevectorSampler()
    job = sampler.run([(qc1,), (qc2,), (qc3,)], shots=100)
    result = job.result()
    
    # Extract by index
    counts1 = result[0].data.c.get_counts()
    counts2 = result[1].data.c.get_counts()
    counts3 = result[2].data.c.get_counts()
    
    return [counts1, counts2, counts3]

# Test your solution
results = challenge_batch()
print("Batch execution results:")
expected = [{'0': 100}, {'1': 100}, {'11': 100}]

all_pass = True
for i, (counts, exp) in enumerate(zip(results, expected)):
    match = list(counts.keys()) == list(exp.keys())
    all_pass = all_pass and match
    status = "✅" if match else "❌"
    print(f"  Circuit {i+1}: {counts} (expected {exp}) {status}")

if all_pass:
    print("\n✅ Challenge 3 PASSED! Batch execution and indexing correct")
else:
    print("\n❌ Challenge 3 FAILED - check your batch format or indexing")

In [ ]:
# ============================================================
# CHALLENGE 4: Error Mitigation Quiz
# ============================================================
# Task: Answer questions about twirling and DD configuration
# This tests your knowledge of runtime options!
# ============================================================

print("=" * 60)
print("CHALLENGE 4: Error Mitigation Knowledge Quiz")
print("=" * 60)

def quiz_error_mitigation():
    """Test knowledge of twirling and DD options."""
    
    questions = [
        {
            "q": "Q1: What is the DEFAULT value of enable_measure for SAMPLER?",
            "options": ["A) True", "B) False", "C) None", "D) 'auto'"],
            "answer": "B",
            "explanation": "Sampler defaults: enable_measure=False (GOMO: Gates Off, Measure Off)"
        },
        {
            "q": "Q2: What is the DEFAULT value of enable_measure for ESTIMATOR?",
            "options": ["A) True", "B) False", "C) None", "D) 'auto'"],
            "answer": "A",
            "explanation": "Estimator defaults: enable_measure=True (different from Sampler!)"
        },
        {
            "q": "Q3: Which DD sequence provides BEST error suppression?",
            "options": ["A) XX", "B) XpXm", "C) XY4", "D) YY"],
            "answer": "C",
            "explanation": "XY4 (X-Y-X-Y) provides superior suppression compared to XX or XpXm"
        },
        {
            "q": "Q4: When does Dynamical Decoupling help MOST?",
            "options": ["A) Short circuits", "B) Circuits with idle qubits", 
                       "C) Single-qubit circuits", "D) Circuits with no barriers"],
            "answer": "B",
            "explanation": "DD inserts pulses during idle periods - no idle time = no benefit"
        },
        {
            "q": "Q5: What is the DEFAULT twirling strategy?",
            "options": ["A) active", "B) active-accum", "C) active-circuit", "D) all"],
            "answer": "B",
            "explanation": "active-accum is the default - twirls all qubits used up to current layer"
        }
    ]
    
    score = 0
    answers = ["B", "A", "C", "B", "B"]  # Your answers
    
    for i, q in enumerate(questions):
        print(f"\n{q['q']}")
        for opt in q['options']:
            print(f"  {opt}")
        
        user_ans = answers[i]  # Change these to test yourself!
        correct = user_ans == q['answer']
        
        if correct:
            score += 1
            print(f"  ✅ Correct! ({q['answer']})")
        else:
            print(f"  ❌ Wrong! Correct: {q['answer']}")
        print(f"  💡 {q['explanation']}")
    
    return score

score = quiz_error_mitigation()
print(f"\n{'=' * 60}")
print(f"Final Score: {score}/5 ({score*20}%)")

if score == 5:
    print("✅ CHALLENGE 4 PASSED! You know your error mitigation options!")
elif score >= 3:
    print("⚠️ Good, but review the defaults again")
else:
    print("❌ Review README Topic 4 (Twirling) and Topic 5 (DD)")

---

## 🎯 Key Takeaways

### Concept Mastery
- ✅ Sampler returns counts/bitstrings, Estimator returns expectation values
- ✅ Sampler **REQUIRES** measurements ("S needs M")
- ✅ PUB format: `[(circuit, params, shots)]` - **TiL** (Tuple in List)
- ✅ Result chain: `result[0].data.meas.get_counts()` - **0-D-M-G**
- ✅ `get_counts()` = histogram dict, `get_bitstrings()` = list of shots
- ✅ Twirling defaults: Sampler has **GOMO** (Gates Off, Measure Off)
- ✅ DD helps only with **idle qubits** ("Defend During Downtime")
- ✅ Qiskit uses **LSB** ordering (rightmost bit = qubit 0)

### Mnemonic Recall Box
```
┌──────────────────────────────────────────────────────────────┐
│ "S needs M" = Sampler needs Measurements                     │
│ "TiL" = Tuple in List [(circuit,)]                          │
│ "CPS" = Circuit, Params, Shots                              │
│ "0-D-M-G" = [0].data.meas.get_counts()                      │
│ "GOMO" = Gates Off, Measure Off (Sampler twirling defaults) │
│ "DD = Defend During Downtime" (idle qubits only)            │
│ "Sampler Samples, Estimator Expects"                         │
└──────────────────────────────────────────────────────────────┘
```

---

## 📚 Resources

### Conceptual Background
- [README.md](./README.md) - Full explanations, traps, mnemonics

### Related Sections
- [Section 4: Run Circuits](../section_4_run_circuits/) - Runtime fundamentals
- [Section 6: Estimator](../section_6_estimator/) - Expectation values
- [Section 7: Results](../section_7_results/) - Advanced result processing

---

**Notebook verified with Qiskit 1.x** | Last updated: December 2025